# Sampling the sequence space for mutations

In biological sequence spaces (DNA, RNA, and proteins), the possible search space is vast and high dimensional. The further a biological sequence mutates away from its original sequence, the more unpredictable the final behavior of the system becomes, with mutations often layering on top of each other at the sequence level in a way that translates non-linearly to the higher order structures - the phenomenon of epistasis. To explore the local mutational landscape required sampling both a well-chosen set of starting sequences and sufficient sampling in their surroundings. 

Finding good starting sequences has a few caveats. The goal of genetic circuits is to actuate some behavior through the interactions of its components in response to a signal. Therefore, sequences that have combinations of weak or strong interactions are desirable. However, the vast majority of sequence space has weak interactions and is thus non-functional, meaning pairs of sequences have extremely weak binding. Finding sequences that can bind to each other strongly is a prerequisite, but the total sampling of the possible circuit space should still be diverse enough to capture many different types of circuits.

For RNA and DNA, complementarity can be used as a way to guarantee binding. Random sampling of the sequence space can be followed up with induced complementarity, for example by delegating one circuit component as a template, then reserving portions of the other components to be complementary to the template strand. The degree and patterning of this induced complementarity can be varied depending on the number of strands and the level of intervention desired. The `SeqGenerator` class therefore has four different ways to generate the components of a genetic circuit, termed 'protocols'. Their differences are demonstrated below.

In [82]:
import sys
import os
import Bio
import numpy as np


if __package__ is None:

    module_path = os.path.abspath(os.path.join('..'))
    sys.path.append(module_path)

    __package__ = os.path.basename(module_path)


from synbio_morpher.utils.common.setup import construct_circuit_from_cfg, prepare_config
from synbio_morpher.utils.data.data_format_tools.common import load_json_as_dict
from synbio_morpher.utils.data.data_format_tools.manipulate_fasta import load_seq_from_FASTA
from synbio_morpher.utils.data.fake_data_generation.seq_generator import RNAGenerator
from synbio_morpher.utils.evolution.evolver import Evolver
from synbio_morpher.utils.misc.type_handling import flatten_listlike, flatten_nested_dict
from synbio_morpher.utils.results.result_writer import ResultWriter


config = load_json_as_dict(os.path.join('..', 'tests', 'configs', 'simple_circuit.json'))


In [83]:
data_writer = ResultWriter(purpose='tests')

In [84]:
paths_circuits = RNAGenerator(data_writer=data_writer).generate_circuits_batch(
            name='toy_RNGA',
            num_circuits=10,
            num_components=3, 
            slength=20,
            proportion_to_mutate=0.5,
            protocol='random',
            template=None)

samples = [None] * 10
for i, p in enumerate(paths_circuits):
    samples[i] = load_seq_from_FASTA(list(p.values())[0], as_type='dict')

In [85]:
samples[0]

{'RNA_0': 'GCCACAAUUAAAGUAGAGGU',
 'RNA_1': 'AUUGUAUAACAUAGGUGUCC',
 'RNA_2': 'CUUAGCUCCAUACAUACGAG'}

In [94]:
from Bio import Align

samples_names = sorted(set(flatten_listlike([list(s.keys()) for s in samples])))
aligner = Align.PairwiseAligner()
ref_aligments = aligner.align(samples[0][samples_names[0]], samples[0][samples_names[0]])
alignments = flatten_listlike(flatten_listlike([[[aligner.align(s[k1], Bio.Seq.complement_rna(s[k2])) for s in samples] for k1 in samples_names] for k2 in samples_names]))
print('Reference alignment (perfect complementarity): ', ref_aligments[0].score)
print('Alignment scores for complements in circuits: ')
print(np.unique([a.score for a in alignments]))
print(alignments[0][0])

Reference alignment (perfect complementarity):  20.0
Alignment scores for complements in circuits: 
[ 7.  8.  9. 10. 11. 12. 13. 14.]
target            0 GCCACAA--U-U-AAAGU---AGAGGU----- 20
                  0 -|-------|-|-||--|---|----|----- 32
query             0 -C-----GGUGUUAA--UUUCA----UCUCCA 20



Now we will compare the random sequence generation to the similarity scores produced by heuristic complementarity-inducing methods.

In [87]:
good_template = 'GCCCCGGGGCUCUCUAUACG'  # toy_mRNA_circuit_133814, ensemble_generate_circuits/2023_02_24_170946/generate_species_templates/circuits
bad_template = 'UAGCCCUUGAUAAGGGCUAA'   # ensemble_generate_circuits/2023_02_24_170946/generate_species_templates/circuits/toy_mRNA_circuit_0.fasta

templates = {'strong': good_template, 'weak': bad_template}
protocols = ['template_mix', 'template_mutate', 'template_split']
path_dict = {}
i = 0
for n, t in templates.items():
    path_dict[n] = {}
    for p in protocols:
        np.random.seed(i)
        i+= 1
        num_circuits = 10 if p == 'template_mutate' else 1
        path_dict[n][p] = RNAGenerator(data_writer=data_writer).generate_circuits_batch(
            name=f'toy_RNA_{n}_{p}',
            num_circuits=num_circuits,
            num_components=3,
            slength=20,
            proportion_to_mutate=0.5,
            protocol=p,
            template=t)

templated_samples = path_dict
for n, v in path_dict.items():
    for prot, paths in v.items():
        templated_samples[n][prot] = [load_seq_from_FASTA(list(pv.values())[0], as_type='dict') for pv in paths]
    # construct_circuit_from_cfg(config)
templated_samples['strong']['template_mix'][:3]


[{'RNA_0': 'GGGCGCGCCCAGUGAAAUCC',
  'RNA_1': 'CCGGCCCGCGUGACAUUUGG',
  'RNA_2': 'CGCGGGCCGGACAGUUAAGC'}]

## Comparison between methods

In [90]:
def convert_seqs_to_binary_complement(refseq, mutseq): 
    return (np.asarray(list(refseq)) == np.asarray(list(mutseq))) * 1

In [105]:
print('Method: template_mutate')
tmut = templated_samples['strong']['template_mutate'][0]
print('Reference sequence: ', templates['strong'])
print('Mutated sequence: ', tmut['RNA_0'], '- pattern: ', ''.join(str(convert_seqs_to_binary_complement(templates['strong'], tmut['RNA_0']))), '\t Mutated: ', round(1- sum(convert_seqs_to_binary_complement(templates['strong'], tmut['RNA_0'])/20),3))
print('Mutated sequence: ', tmut['RNA_1'], '- pattern: ', ''.join(str(convert_seqs_to_binary_complement(templates['strong'], tmut['RNA_1']))), '\t Mutated: ', round(1- sum(convert_seqs_to_binary_complement(templates['strong'], tmut['RNA_1'])/20),3))
print('Mutated sequence: ', tmut['RNA_2'], '- pattern: ', ''.join(str(convert_seqs_to_binary_complement(templates['strong'], tmut['RNA_2']))), '\t Mutated: ', round(1- sum(convert_seqs_to_binary_complement(templates['strong'], tmut['RNA_2'])/20),3))

Method: template_mutate
Reference sequence:  GCCCCGGGGCUCUCUAUACG
Mutated sequence:  GCCCCUGGGCUCCAAAUACG - pattern:  [1 1 1 1 1 0 1 1 1 1 1 1 0 0 0 1 1 1 1 1] 	 Mutated:  0.2
Mutated sequence:  UGAACGUCGCUCACUAUAAG - pattern:  [0 0 0 0 1 1 0 0 1 1 1 1 0 1 1 1 1 1 0 1] 	 Mutated:  0.4
Mutated sequence:  GCCCAGGGGACAUGUACAUG - pattern:  [1 1 1 1 0 1 1 1 1 0 0 0 1 0 1 1 0 1 0 1] 	 Mutated:  0.35


In [106]:
print('Method: template_mix')
tmix = templated_samples['strong']['template_mix'][0]
print('Reference sequence: ', templates['strong'])
print('Mix sequence: ', tmix['RNA_0'], '- pattern: ', ''.join(str(convert_seqs_to_binary_complement(templates['strong'], tmix['RNA_0']))), '\t Mutated: ', round(1- sum(convert_seqs_to_binary_complement(templates['strong'], tmix['RNA_0'])/20),3))
print('Mix sequence: ', tmix['RNA_1'], '- pattern: ', ''.join(str(convert_seqs_to_binary_complement(templates['strong'], tmix['RNA_1']))), '\t Mutated: ', round(1- sum(convert_seqs_to_binary_complement(templates['strong'], tmix['RNA_1'])/20),3))
print('Mix sequence: ', tmix['RNA_2'], '- pattern: ', ''.join(str(convert_seqs_to_binary_complement(templates['strong'], tmix['RNA_2']))), '\t Mutated: ', round(1- sum(convert_seqs_to_binary_complement(templates['strong'], tmix['RNA_2'])/20),3))

Method: template_mix
Reference sequence:  GCCCCGGGGCUCUCUAUACG
Mix sequence:  GGGCGCGCCCAGUGAAAUCC - pattern:  [1 0 0 1 0 0 1 0 0 1 0 0 1 0 0 1 0 0 1 0] 	 Mutated:  0.65
Mix sequence:  CCGGCCCGCGUGACAUUUGG - pattern:  [0 1 0 0 1 0 0 1 0 0 1 0 0 1 0 0 1 0 0 1] 	 Mutated:  0.65
Mix sequence:  CGCGGGCCGGACAGUUAAGC - pattern:  [0 0 1 0 0 1 0 0 1 0 0 1 0 0 1 0 0 1 0 0] 	 Mutated:  0.7


In [107]:
print('Method: template_split')
tsplit = templated_samples['strong']['template_split'][0]
print('Reference sequence: ', templates['strong'])
print('Split sequence: ', tsplit['RNA_0'], '- pattern: ', ''.join(str(convert_seqs_to_binary_complement(templates['strong'], tsplit['RNA_0']))), '\t Mutated: ', round(1- sum(convert_seqs_to_binary_complement(templates['strong'], tsplit['RNA_0'])/20),3))
print('Split sequence: ', tsplit['RNA_1'], '- pattern: ', ''.join(str(convert_seqs_to_binary_complement(templates['strong'], tsplit['RNA_1']))), '\t Mutated: ', round(1- sum(convert_seqs_to_binary_complement(templates['strong'], tsplit['RNA_1'])/20),3))
print('Split sequence: ', tsplit['RNA_2'], '- pattern: ', ''.join(str(convert_seqs_to_binary_complement(templates['strong'], tsplit['RNA_2']))), '\t Mutated: ', round(1- sum(convert_seqs_to_binary_complement(templates['strong'], tsplit['RNA_2'])/20),3))

Method: template_split
Reference sequence:  GCCCCGGGGCUCUCUAUACG
Split sequence:  CGGGGCGGGCUCUCUAUACG - pattern:  [0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1] 	 Mutated:  0.3
Split sequence:  GCCCCGCCCGAGUCUAUACG - pattern:  [1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 1 1] 	 Mutated:  0.3
Split sequence:  GCCCCGGGGCUCAGAUAUCG - pattern:  [1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 1 1] 	 Mutated:  0.3
